In [ ]:
import pandas as pd
from sklearn.mixture import BayesianGaussianMixture
from sklearn.decomposition import PCA
import numpy as np
import pickle
import json


In [ ]:
international_regions = [
'vct_pacific',
'vct_emea',
'vct_china',
'vct_americas'
]

changers_regions = [
'game_changers_kr',
'game_changers_sa',
'game_changers_jpn',
'game_changers_latam',
'game_changers_na',
'game_changers_emea',
'game_changers_pacific',
'game_changers_east_asia',
'game_changers_sea'
]


# (game-changers, vct-international, vct-challengers)
LEAGUE = "game-changers"

# (2022, 2023, 2024)
YEAR = 2024

In [ ]:
agg_df = pd.read_pickle(f'/content/{LEAGUE}-{YEAR}-agg.pkl')
df = pd.read_pickle(f'/content/{LEAGUE}-{YEAR}-df.pkl')

with open(f'/content/vct-bggm.pkl', 'rb') as f:
    bgmm = pickle.load(f)

with open(f'/content/vct-pca.pkl', 'rb') as f:
    pca = pickle.load(f)

with open(f'/content/players.json', 'r') as f:
    player_df = pd.DataFrame(json.load(f))

with open(f'/content/leagues.json', 'r') as f:
    leagues_df = pd.DataFrame(json.load(f))

with open(f'/content/teams.json', 'r') as f:
    teams_df = pd.DataFrame(json.load(f))

leagues_df.head()

,league_id,region,dark_logo_url,light_logo_url,name,slug
0,109029777807406730,KR,http://static.lolesports.com/leagues/166366237...,http://static.lolesports.com/leagues/166366237...,game_changers_kr,game_changers_kr
1,112609302000026507,INTL,http://static.lolesports.com/leagues/171828157...,http://static.lolesports.com/leagues/171828157...,game_changers_sa,game_changers_sa
2,108752229027041361,JP,http://static.lolesports.com/leagues/165942731...,http://static.lolesports.com/leagues/165942731...,game_changers_jpn,game_changers_jpn
3,106375817979489820,BR,http://static.lolesports.com/leagues/logo-gc-p...,http://static.lolesports.com/leagues/logo-gc-p...,game_changers_series_brazil,game_changers_series_brazil
4,109879575186437267,LATAM,http://static.lolesports.com/leagues/167662925...,http://static.lolesports.com/leagues/167662925...,game_changers_latam,game_changers_latam


In [ ]:
if LEAGUE == 'vct-international':
    print('detected international')
    leagues_df = leagues_df.loc[leagues_df['slug'].isin(international_regions)]
elif LEAGUE == 'game-changers':
    print('detected game changers')
    leagues_df = leagues_df.loc[leagues_df['slug'].isin(changers_regions)]
region_list = leagues_df.loc[:, 'name'].drop_duplicates().to_list()
print(region_list)

detected game changers
['game_changers_kr', 'game_changers_sa', 'game_changers_jpn', 'game_changers_latam', 'game_changers_na', 'game_changers_emea', 'game_changers_pacific', 'game_changers_east_asia', 'game_changers_sea']


In [ ]:
if LEAGUE == 'vct-challengers':
    print('detected challengers')
    rm = {
        'challengers_sea_id': 'challengers_pacific',
        'challengers_sea_vn': 'challengers_pacific',
        'challengers_sea_sg_and_my': 'challengers_pacific',
        'challengers_br': 'challengers_americas',
        'challengers_latam': 'challengers_americas',
        'challengers_na': 'challengers_americas',
        'challengers_italy': 'challengers_emea',
        'challengers_latam_n': 'challengers_americas',
        'challengers_sea_ph': 'challengers_pacific',
        'challengers_south_asia': 'challengers_pacific',
        'challengers_portugal': 'challengers_emea',
        'challengers_sea_hk_and_tw': 'challengers_pacific',
        'challengers_jpn': 'challengers_pacific',
        'challengers_latam_s': 'challengers_americas',
        'challengers_sea_th': 'challengers_pacific',
        'challengers_apac': 'challengers_pacific',
        'challengers_kr': 'challengers_pacific'
    }
    region_list = ['challengers_pacific', 'challengers_americas', 'challengers_emea']
elif LEAGUE == 'game-changers':
    print('detected game changers')
    rm = {
        'game_changers_kr': 'game_changers_pacific',
        'game_changers_sa': 'game_changers_pacific',
        'game_changers_jpn': 'game_changers_pacific',
        'game_changers_latam': 'game_changers_americas',
        'game_changers_na': 'game_changers_americas',
        'game_changers_emea': 'game_changers_emea',
        'game_changers_pacific': 'game_changers_pacific',
        'game_changers_east_asia': 'game_changers_pacific',
        'game_changers_sea': 'game_changers_pacific'
    }
    region_list = ['game_changers_pacific', 'game_changers_americas', 'game_changers_emea']
else:
    rm = {r:r for r in region_list}

detected game changers


In [ ]:
player_df.head()

,id,handle,first_name,last_name,status,photo_url,home_team_id,created_at,updated_at
0,107025876564296044,Toma,Tommi,Lehtinen,active,http://static.lolesports.com/players/163303303...,106652238530500027,2021-10-01T11:19:00Z,2021-10-01T11:19:00Z
1,106977714184829995,Stefanie,Stefanie,Jones,active,http://static.lolesports.com/players/163235037...,106976771652907805,2021-09-22T22:41:26Z,2023-04-09T14:50:55Z
2,107176790303905255,aRth,Arthur,Hardman,active,None,107174817570297246,2021-10-28T14:16:57Z,2021-11-01T13:52:31Z
3,107282855606202828,godwana,Engin,Balcı,active,http://static.lolesports.com/players/163700646...,107021298845350518,2021-11-15T21:00:23Z,2023-05-29T08:05:32Z
4,106977742188901804,Jaxsen,Jackson,Popelka,active,http://static.lolesports.com/players/163235080...,106976774374552352,2021-09-22T22:49:42Z,2021-09-22T22:49:42Z


In [ ]:
regions_df = player_df.merge(teams_df,
                           left_on='home_team_id',
                           right_on='id',
                           how='inner')
regions_df = regions_df.merge(leagues_df,
                        left_on='home_league_id',
                        right_on='league_id',
                        how='inner').sort_values('updated_at', ascending=False)
regions_df = regions_df.sort_values('updated_at', ascending=False).loc[:, ['id_x', 'slug_y']].drop_duplicates('id_x')
regions_df.head()

,id_x,slug_y
3827,113068487057105034,game_changers_emea
1123,113068485580341633,game_changers_emea
648,113038968137007776,game_changers_emea
4399,110196702294187829,game_changers_emea
4156,110196509462001155,game_changers_emea


In [ ]:
df.head()

,ROUND_NUMBER,OUTCOME,SIDE,KILLS,DEATHS,ASSISTS,COMBAT_SCORE,KILLS_STINGER,KILLS_BUCKY,KILLS_JUDGE,...,YORU_PICK_RATE,ISO_PICK_RATE,CLOVE_PICK_RATE,VYSE_PICK_RATE,DUELIST_PICK_RATE,INITIATOR_PICK_RATE,SENTINEL_PICK_RATE,CONTROLLER_PICK_RATE,id,metadata
0,1.0,1.0,-1.0,1.0,0.0,0.0,230.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,110196509462001155,"{map=LOTUS, game_id=val:00d5b763-f082-481f-8d7..."
1,1.0,1.0,-1.0,2.0,0.0,1.0,354.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,107025876838906695,"{map=LOTUS, game_id=val:00d5b763-f082-481f-8d7..."
2,1.0,1.0,-1.0,1.0,1.0,0.0,250.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,110196607873067874,"{map=LOTUS, game_id=val:00d5b763-f082-481f-8d7..."
3,1.0,1.0,-1.0,1.0,1.0,0.0,234.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,110196702294187829,"{map=LOTUS, game_id=val:00d5b763-f082-481f-8d7..."
4,1.0,1.0,-1.0,0.0,0.0,1.0,56.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,110196647859540239,"{map=LOTUS, game_id=val:00d5b763-f082-481f-8d7..."


In [ ]:
agg_df.drop(agg_df.index[:1], inplace=True)
agg_df.head()

,ROUND_NUMBER,SIDE,DEATHS,ASSISTS,KILLS_STINGER,KILLS_BUCKY,KILLS_JUDGE,KILLS_SPECTRE,KILLS_BULLDOG,KILLS_GUARDIAN,...,RAZE_PICK_RATE,REYNA_PICK_RATE,SAGE_PICK_RATE,SKYE_PICK_RATE,SOVA_PICK_RATE,VIPER_PICK_RATE,YORU_PICK_RATE,ISO_PICK_RATE,CLOVE_PICK_RATE,VYSE_PICK_RATE
id,,,,,,,,,,,,,,,,,,,,,
106371570922864110,0.740826,-0.450660,-0.809343,0.791967,-0.770870,-0.364931,-0.524846,0.371144,-0.929466,-1.237198,...,-0.518179,-0.246961,-0.276105,-0.450332,1.789826,-0.660963,-0.257586,-0.178576,-0.23074,0.0
106371572811742769,1.363058,-0.605352,-0.519918,0.184979,0.963865,-0.364931,-0.524846,-0.575765,0.676827,-0.845262,...,-0.518179,-0.246961,-0.276105,-0.450332,1.590935,1.619483,-0.257586,-0.178576,-0.23074,0.0
106371583096241717,0.132270,-0.251934,-0.674601,0.479166,0.836918,-0.364931,-0.524846,0.951732,-0.122018,-1.237198,...,-0.518179,-0.246961,-0.276105,-0.450332,1.778474,0.658654,-0.257586,-0.178576,-0.23074,0.0
106371585325121013,-0.753410,-3.218659,-0.484139,-1.665384,-0.770870,-0.364931,-0.524846,-1.061927,-1.451732,0.402176,...,-0.518179,-0.246961,-0.276105,-0.450332,1.890900,-0.660963,-0.257586,-0.178576,-0.23074,0.0
106371625243295242,0.740826,0.023797,0.362234,0.624839,-0.770870,-0.364931,-0.524846,1.203417,-0.940189,1.047520,...,-0.518179,-0.246961,-0.276105,2.330316,1.690366,-0.660963,-0.257586,-0.178576,-0.23074,0.0


In [ ]:
target = '106371570922864110'
print(list(agg_df.loc[target].values))
print(bgmm.predict_proba(agg_df.loc[target].values.reshape(1,-1)))
print(bgmm.predict(agg_df.loc[target].values.reshape(1,-1)))
print(pca.transform(agg_df.loc[target].values.reshape(1,-1)))
player_df.loc[player_df['id'] == target].drop_duplicates('id')

[0.7408262791575616, -0.4506602420923629, -0.8093430974180001, 0.7919667222470715, -0.7708696985200681, -0.36493104904622786, -0.5248455724144154, 0.37114439411051664, -0.929465823523418, -1.2371975467085878, 1.908716305723392, -2.204851668785134, -0.40090768660019893, -0.568141424886023, -0.6900462449753433, -0.2198597432741583, -0.48908182448679466, -1.0993992574614118, -0.5250299267537103, -0.5174534748169473, -0.9469316735017599, -0.36476721698154585, -0.29047298713753417, -0.20281551440678144, -0.8866252848615821, -1.5615407629319455, -1.2665377429101485, -0.13682018290809, 0.027901959993732996, -0.2957734234600459, 1.937015259152714, -0.3942425755436335, -0.2034816850612795, -0.52142605164717, -0.2337182836758459, 2.3844802541743326, -0.49904686546433913, -0.23966040517718784, -0.5224143223105724, 0.0, -0.5648471555216513, -0.24728858272955603, -0.6252360499025587, -0.18093189121130213, -0.5181789279342509, -0.24696067188218532, -0.2761049032020882, -0.4503322475414043, 1.7898257

,id,handle,first_name,last_name,status,photo_url,home_team_id,created_at,updated_at
2089,106371570922864110,tayhuhu,Taynah,Faria,active,None,109993966808522027,2023-03-09T15:16:33Z,2024-07-16T02:10:01Z


In [ ]:
X = agg_df.to_numpy()
pred = bgmm.predict(X)


In [ ]:
import random
from collections import Counter
def weightedRandom(max):
    numDice = 5
    num = 0;
    for _ in range(numDice):
        num += random.random() * (max/numDice);
    return round(num);


dualists = {r:[] for r in region_list}
print(region_list)
print(dualists)
for i, id in enumerate(list(agg_df.index)):
    if pred[i] == 4 or pred[i] == 2:
        pc0 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][0]
        pc3 = -1 * pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][3]
        r = rm[regions_df.loc[regions_df['id_x'] == id, 'slug_y'].head(1).values[0]]
        print(r)
        weight = 0.6 * pc0 + 0.3 * pc3
        # if pc3 > -1:
        print(list(player_df.loc[player_df['id'] == id, ['id', 'handle']].drop_duplicates('id').values[0]), weight)
        dualists[r].append((list(player_df.loc[player_df['id'] == id, ['id', 'handle']].drop_duplicates('id').values[0]), weight))

print(weightedRandom(15))
print(dualists)
duelist_pdf = {r:{} for r in region_list}
if LEAGUE == 'game-changers':
    print('detected game changers')
    dualists['game_changers_emea'] = [(['110196698617452805', 'rezq'], 1)]
for r in region_list:
    print(r)
    dualists[r] = sorted(dualists[r], key=lambda x: x[1], reverse=True)
    pcaw_tot = sum([d[1] for d in dualists[r]])
    dualists[r] = list(map(lambda x: (*x[0], (x[1]/pcaw_tot)*100), dualists[r]))
    for d in dualists[r]:
        print(d)

    print("-----")
    d_counter = list()
    for _ in range(1000):
        d_counter.append(dualists[r][weightedRandom(len(dualists[r])-1)][0])
    duelist_exp = Counter(d_counter)
    total_count = sum(duelist_exp.values())
    duelist_pdf[r] = { k: v/total_count for k,v in Counter(d_counter).items()}
    duelist_pdf[r] = {k:v for k, v in sorted(duelist_pdf[r].items(), key=lambda x: x[1], reverse=True)}
print(json.dumps(duelist_pdf, indent=4))

['game_changers_pacific', 'game_changers_americas', 'game_changers_emea']
{'game_changers_pacific': [], 'game_changers_americas': [], 'game_changers_emea': []}
game_changers_pacific
['107115831389167224', 'enerii'] 0.8484220244497883
game_changers_americas
['107600768817092518', 'Zaki2d'] 1.3518142245332423
game_changers_americas
['111170144102629356', 'xann'] 4.384686219982534
game_changers_americas
['112127772356148609', 'natas'] -0.5713537859630547
game_changers_pacific
['112184651889340927', 'Yanni'] 1.4127279620726383
8
{'game_changers_pacific': [(['107115831389167224', 'enerii'], 0.8484220244497883), (['112184651889340927', 'Yanni'], 1.4127279620726383)], 'game_changers_americas': [(['107600768817092518', 'Zaki2d'], 1.3518142245332423), (['111170144102629356', 'xann'], 4.384686219982534), (['112127772356148609', 'natas'], -0.5713537859630547)], 'game_changers_emea': []}
detected game changers
game_changers_pacific
('112184651889340927', 'Yanni', 62.47829513712918)
('1071158313891

In [ ]:
regions_df.loc[regions_df['id_x'] == id, 'slug_y'].head()

,slug_y


In [ ]:
sentinals = {r:[] for r in region_list}

for i, id in enumerate(list(agg_df.index)):
    if pred[i] == 3 or pred[i] == 0:
        pc0 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][0]
        pc1 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][1]
        pc2 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][2]
        pc23 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][23]
        if len(regions_df.loc[regions_df['id_x'] == id, 'slug_y'].head(1).values) <= 0:
            continue
        r = rm[regions_df.loc[regions_df['id_x'] == id, 'slug_y'].head(1).values[0]]

        if pc0 < 4 and pc1 < 0 and pc2 > 0 and pc1 < pc0:
            weight = abs(-0.75 - pc23)
            sentinals[r].append((list(player_df.loc[player_df['id'] == id, ['id', 'handle']].drop_duplicates('id').values[0]), weight))

sentinals_pdf = {r:{} for r in region_list}
for r in region_list:
    sentinals[r] = sorted(sentinals[r], key=lambda x: x[1])[:(len(sentinals[r])//3)]
    print(sentinals[r])
    pcaw_tot = sum([d[1] for d in sentinals[r]])
    pcaw_max = max([s[1] for s in sentinals[r]])
    sentinals[r] = list(map(lambda x: (*x[0], ((pcaw_max - x[1])/pcaw_tot)*100), sentinals[r]))
    for s in sentinals[r]:
        print(s)
    print("-----")
    total_count = sum([s[2] for s in sentinals[r]])
    sentinals_pdf[r] = {s[0]: s[2]/total_count for s in sentinals[r]}
    sentinals_pdf[r] = {k:v for k, v in sorted(sentinals_pdf[r].items(), key=lambda x: x[1], reverse=True)}
print(json.dumps(sentinals_pdf, indent=4))

[(['112829095245438687', 'atti'], 0.016803480764360224), (['111067958377934403', 'SereNa1'], 0.025721330726282465), (['112184598848565742', 'bles'], 0.03748972756320301), (['109030070859531583', 'HakuNa'], 0.0577541279756727), (['107115868964705117', 'Envious'], 0.11025424325284339), (['107115853451678535', 'crescents'], 0.1454820633513414), (['112439973913655736', 'f1yk'], 0.22813200053161686), (['112748512170080589', 'Neko'], 0.291386254502597)]
('112829095245438687', 'atti', 30.07401839478363)
('111067958377934403', 'SereNa1', 29.097279831961607)
('112184598848565742', 'bles', 27.808331591938142)
('109030070859531583', 'HakuNa', 25.58884803706352)
('107115868964705117', 'Envious', 19.838707884138024)
('107115853451678535', 'crescents', 15.980337254302604)
('112439973913655736', 'f1yk', 6.928000513553946)
('112748512170080589', 'Neko', 0.0)
-----
[(['107025876448349386', 'Noia'], 0.08321030652842587), (['112812064723415288', 'MayziN'], 0.19280768122078074), (['109993752630503080', 'b

In [ ]:
controller = {r:[] for r in region_list}

for i, id in enumerate(list(agg_df.index)):
    if pred[i] == 3:
        pc0 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][0]
        pc2 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][2]
        pc12 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][12]
        if len(regions_df.loc[regions_df['id_x'] == id, 'slug_y'].head(1).values) <= 0:
            continue
        r = rm[regions_df.loc[regions_df['id_x'] == id, 'slug_y'].head(1).values[0]]

        if pc2 <= 0:
            weight = abs(-0.25 - pc12)
            controller[r].append((list(player_df.loc[player_df['id'] == id, ['id', 'handle']].drop_duplicates('id').values[0]), weight))


controller_pdf = {r:{} for r in region_list}
for r in region_list:
    controller[r] = sorted(controller[r], key=lambda x: x[1])[:(len(controller[r])//3)]
    pcaw_tot = sum([d[1] for d in controller[r]])
    pcaw_max = max([s[1] for s in controller[r]])
    controller[r] = list(map(lambda x: (*x[0], ((pcaw_max - x[1])/pcaw_tot)*100), controller[r]))
    for s in controller[r]:
        print(s)
    print("-----")
    total_count = sum([s[2] for s in controller[r]])
    controller_pdf[r] = {s[0]: s[2]/total_count for s in controller[r]}
    controller_pdf[r] = {k:v for k, v in sorted(controller_pdf[r].items(), key=lambda x: x[1], reverse=True)}
print(json.dumps(controller_pdf, indent=4))

('111007890889417907', 'Yxx', 13.520281477537216)
('112279748241553126', 'Kokasasuoka', 12.50290608768532)
('111007786107165592', 'B1nby0', 11.476043808009456)
('112173250539931247', 'americano', 10.159838805640579)
('112829091445137105', 'YS2', 6.650731377010759)
('108764513807403457', 'R4M', 5.889537385863124)
('111057577376358736', 'yunah', 5.599478346177344)
('112173255577735239', 'pop0', 3.9269469300670075)
('112184651822196661', 'Bugsy', 3.7314466802265494)
('108748099637064824', 'suzu2', 2.400522748643737)
('112173224518665813', 'CasperR', 1.331620219223903)
('111007943120168151', 'h4ste', 0.7183631821019975)
('111057371424123924', 'UR', 0.0)
-----
('106978689358558927', 'cbui', 14.746362815155988)
('112812064548411043', 'Junki', 13.672244563490644)
('112127657431166971', 'meL3', 13.568355482949235)
('112127711572007165', 'Luwucia', 12.274258654559073)
('107897750670292594', 'Lied', 9.234294692273942)
('106977706805672993', 'theia', 7.7497329484321975)
('112127736086494559', 'iY

In [ ]:
initiator = {r:[] for r in region_list}

for i, id in enumerate(list(agg_df.index)):
    if pred[i] == 1:
        pc1 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][1]
        pc3 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][3]
        if len(regions_df.loc[regions_df['id_x'] == id, 'slug_y'].head(1).values) <= 0:
            continue
        r = rm[regions_df.loc[regions_df['id_x'] == id, 'slug_y'].head(1).values[0]]
        if pc1 >= 1.5:
            weight = abs(-1.25 - pc3)

            initiator[r].append((list(player_df.loc[player_df['id'] == id, ['id', 'handle']].drop_duplicates('id').values[0]), weight))

initiator_pdf = {r:{} for r in region_list}
for r in region_list:
    initiator[r] = sorted(initiator[r], key=lambda x: x[1])[:(len(initiator[r])//2)]
    pcaw_tot = sum([d[1] for d in initiator[r]])
    pcaw_max = max([s[1] for s in initiator[r]])
    initiator[r] = list(map(lambda x: (*x[0], ((pcaw_max - x[1])/pcaw_tot)*100), initiator[r]))
    for s in initiator[r]:
        print(s)
    print("-----")
    total_count = sum([s[2] for s in initiator[r]])
    initiator_pdf[r] = {s[0]: s[2]/total_count for s in initiator[r]}
    initiator_pdf[r] = {k:v for k, v in sorted(initiator_pdf[r].items(), key=lambda x: x[1], reverse=True)}
print(json.dumps(initiator_pdf, indent=4))

('109030016997304064', 'YamzzI', 16.72831105660396)
('108748092183458913', 'Healing', 15.080522838178018)
('110745687059117485', 'maripo', 15.030885075372789)
('112748583754765021', 'miyaJP', 11.413491508078017)
('112279753081852283', 'shiye', 9.987034179958734)
('112439969847428557', 'enxen', 1.903646032897728)
('112184585365415828', 'MicroBrilk', 1.191701027886636)
('108748093144321973', 'aco', 0.9872661092646231)
('110768385712447998', 'tub', 0.0)
-----
('109881533659576993', 'ChloeZerA', 22.72782543076752)
('108449806078327743', 'poniX', 18.769504664724778)
('109881375073356472', 'AlexaIsHot', 16.93969879342492)
('106977489224907631', 'LunaFox', 13.96878128404963)
('107600817233778673', 'Andrehs', 10.504677481965375)
('108449803586045929', 'Jia', 10.18822653353654)
('112127708838064181', 'May1', 5.019719943827554)
('109881268359290683', 'Dano', 0.0)
-----
('107025877954628831', 'eva', 30.23094147119283)
('107025876156414780', '6sonya', 22.444572405278244)
('111006824117215933', 'to

In [ ]:
target = '106371570922864110'
stripped_meta_df = df.loc[:, ~df.columns.isin(['metadata'])]
agg_plus_df = stripped_meta_df.groupby('id').agg({
    'ROUND_NUMBER': 'max',
    **{col: 'mean' for col in stripped_meta_df.columns if col not in ['id', 'ROUND_NUMBER']}
})
agg_plus_df.drop(agg_plus_df.index[:1], inplace=True)
agg_plus_df['MAX_PICK_RATE'] = agg_plus_df[['CONTROLLER_PICK_RATE', 'SENTINEL_PICK_RATE', 'INITIATOR_PICK_RATE', 'DUELIST_PICK_RATE']].max(axis=1)
agg_plus_df.head()
# agg_plus_df.loc[target, ['CONTROLLER_PICK_RATE', 'SENTINEL_PICK_RATE', 'INITIATOR_PICK_RATE', 'DUELIST_PICK_RATE']]
print(agg_plus_df.loc[target, ['MAX_PICK_RATE']])
player_df.loc[player_df['id'] == target].drop_duplicates('id')

MAX_PICK_RATE    1.0
Name: 106371570922864110, dtype: float64


,id,handle,first_name,last_name,status,photo_url,home_team_id,created_at,updated_at
2089,106371570922864110,tayhuhu,Taynah,Faria,active,None,109993966808522027,2023-03-09T15:16:33Z,2024-07-16T02:10:01Z


In [ ]:
flex_pool = agg_plus_df.loc[agg_plus_df['MAX_PICK_RATE'] <= 0.65, ['MAX_PICK_RATE']].sort_values(by='MAX_PICK_RATE', ascending=False).to_dict()['MAX_PICK_RATE']
flex_pool_map = player_df.loc[player_df['id'].isin(list(flex_pool.keys())), ['id', 'handle']].drop_duplicates('id').set_index('id').to_dict()['handle']
flex = {r:[] for r in region_list}

for i, id in enumerate(list(flex_pool_map.keys())):
    pc1 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][1]
    pc23 = pca.transform(agg_df.loc[id].values.reshape(1,-1))[0][23]
    if len(regions_df.loc[regions_df['id_x'] == id, 'slug_y'].head(1).values) <= 0:
        continue
    r = rm[regions_df.loc[regions_df['id_x'] == id, 'slug_y'].head(1).values[0]]
    if pc1 >= 1:
        weight = abs(-0.9 - pc23)

        flex[r].append((list(player_df.loc[player_df['id'] == id, ['id', 'handle']].drop_duplicates('id').values[0]), weight))

flex_pdf = {r:{} for r in region_list}
for r in region_list:
    flex[r] = sorted(flex[r], key=lambda x: x[1])[:(len(flex[r])//2)]
    pcaw_tot = sum([f[1] for f in flex[r]])
    pcaw_max = max([f[1] for f in flex[r]])
    flex[r] = list(map(lambda x: (*x[0], ((pcaw_max - x[1])/pcaw_tot)*100), flex[r]))
    for f in flex[r]:
        print(f)
    print("-----")
    total_count = sum([f[2] for f in flex[r]])
    flex_pdf[r] = {f[0]: f[2]/total_count for f in flex[r]}
    flex_pdf[r] = {k:v for k, v in sorted(flex_pdf[r].items(), key=lambda x: x[1], reverse=True)}
print(json.dumps(flex_pdf, indent=4))

('111007815983932579', '332', 52.926517324159015)
('111057371424123924', 'UR', 50.34690321637177)
('111007889682488230', 'BIBI', 46.19856226303841)
('112184676058720186', 'ArinJi', 35.301637112304505)
('111057597544669531', 'Z1an', 0.0)
-----
('112127605393420211', 'bae1', 0.0)
-----
('107691411227382221', 'Looxie', 0.0)
-----
{
    "game_changers_pacific": {
        "111007815983932579": 0.28643979237001543,
        "111057371424123924": 0.2724788486543393,
        "111007889682488230": 0.250027911365661,
        "112184676058720186": 0.19105344760998416,
        "111057597544669531": 0.0
    },
    "game_changers_americas": {
        "112127605393420211": NaN
    },
    "game_changers_emea": {
        "107691411227382221": NaN
    }
}


<ipython-input-180-915cf1cadffb>:26: RuntimeWarning: invalid value encountered in scalar divide
  flex_pdf[r] = {f[0]: f[2]/total_count for f in flex[r]}


In [ ]:
final_pdf = dict()
final_pdf['duelist'] = duelist_pdf
final_pdf['sentinel'] = sentinals_pdf
final_pdf['controller'] = controller_pdf
final_pdf['flex'] = flex_pdf
final_pdf['initiator'] = initiator_pdf
with open(f'/content/{LEAGUE}-{YEAR}-pdfs.json', 'w') as f:
    json.dump(final_pdf, f, indent=4)